In [31]:
import duckdb 
connection = duckdb.connect()

In [32]:
import os
data_folder = 'data'

csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

relations = {}

for file in csv_files:
    file_path = os.path.join(data_folder, file)
    rel = duckdb.read_csv(file_path)
    relations[file] = rel

Now that all the csv files are properly loaded, raw data will be processed into useful tabular data.

### Data profiling

In [33]:
import pandas as pd

'''
This function extracts summary statistics 
and other descriptive values for datasets
'''

def data_profiling(dataset_path):
    # Load CSV into a Pandas DataFrame
    df = pd.read_csv(dataset_path)
    # Features (columns)
    print("Features:\n", df.columns.tolist())
    
    # Number of rows
    print(f"Number of rows: {len(df)}")
    
    # Summary statistics for each column
    for col in df.columns:
        print(f"\nFeature: {col}\n{df[col].describe()}")
    
    # Missing values
    print("\nMissing values per column:\n", df.isnull().sum())
    
    # Unique values
    print("\nUnique values per column:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()} unique values")
        
    # Duplicate rows
    print("\nDuplicate rows:", df.duplicated().sum())

In [17]:
datasets = [dataset for dataset in relations]
datasets

['circuits.csv',
 'status.csv',
 'lap_times.csv',
 'sprint_results.csv',
 'drivers.csv',
 'races.csv',
 'constructors.csv',
 'constructor_standings.csv',
 'qualifying.csv',
 'driver_standings.csv',
 'constructor_results.csv',
 'pit_stops.csv',
 'seasons.csv',
 'results.csv']

### Circuits

In [18]:
circuits = pd.read_csv("data/circuits.csv")
data_profiling("data/circuits.csv")

Features:
 ['circuitId', 'circuitRef', 'name', 'location', 'country', 'lat', 'lng', 'alt', 'url']
Number of rows: 77

Feature: circuitId
count    77.000000
mean     39.883117
std      23.001701
min       1.000000
25%      20.000000
50%      40.000000
75%      59.000000
max      80.000000
Name: circuitId, dtype: float64

Feature: circuitRef
count              77
unique             77
top       albert_park
freq                1
Name: circuitRef, dtype: object

Feature: name
count                                 77
unique                                77
top       Albert Park Grand Prix Circuit
freq                                   1
Name: name, dtype: object

Feature: location
count            77
unique           75
top       Barcelona
freq              2
Name: location, dtype: object

Feature: country
count      77
unique     35
top       USA
freq       11
Name: country, dtype: object

Feature: lat
count    77.000000
mean     33.442925
std      22.808866
min     -37.849700
25%      32

Every circuit has a unique id and name, location, and its own latitude, longitude and altitude, the latter which is aerodinamically important and contextualizes the model.

All values seem to be within reasonable boundaries, with correct datatypes and consistent counts.

Some altitude and location values are shared, which makes sense since many tracks share similar altitudes due to proximity.

In [19]:
alt_counts = circuits['alt'].value_counts()
for alt_value in alt_counts.index:
    count = alt_counts[alt_value]
    if count > 1:
        matching_rows = circuits[circuits['alt'] == alt_value][['name', 'country']]
        for _, row in matching_rows.iterrows():
            print(f"{count} {row['name']} ({row['country']})")

3 Sepang International Circuit (Malaysia)
3 Marina Bay Street Circuit (Singapore)
3 Sebring International Raceway (USA)
2 Autodromo Enzo e Dino Ferrari (Italy)
2 Circuito de Jerez (Spain)
2 Silverstone Circuit (UK)
2 Scandinavian Raceway (Sweden)
2 Korean International Circuit (Korea)
2 Miami International Autodrome (USA)
2 Donington Park (UK)
2 Reims-Gueux (France)
2 Prince George Circuit (South Africa)
2 Jeddah Corniche Circuit (Saudi Arabia)
2 Fair Park (USA)
2 Nivelles-Baulers (Belgium)
2 Long Beach (USA)
2 Losail International Circuit (Qatar)
2 Bahrain International Circuit (Bahrain)
2 Circuit de Monaco (Monaco)
2 Istanbul Park (Turkey)
2 Autódromo do Estoril (Portugal)


### Laptimes

In [20]:
laptimes = pd.read_csv("data/lap_times.csv")
data_profiling("data/lap_times.csv")

Features:
 ['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds']
Number of rows: 575989

Feature: raceId
count    575989.000000
mean        588.321155
std         431.564765
min           1.000000
25%         137.000000
50%         856.000000
75%         994.000000
max        1132.000000
Name: raceId, dtype: float64

Feature: driverId
count    575989.000000
mean        316.035923
std         384.567928
min           1.000000
25%          16.000000
50%          44.000000
75%         821.000000
max         860.000000
Name: driverId, dtype: float64

Feature: lap
count    575989.000000
mean         30.014316
std          18.412948
min           1.000000
25%          14.000000
50%          29.000000
75%          44.000000
max          87.000000
Name: lap, dtype: float64

Feature: position
count    575989.000000
mean          9.659827
std           5.531333
min           1.000000
25%           5.000000
50%           9.000000
75%          14.000000
max          24.000000
Name: pos

In [21]:
threshold = laptimes["milliseconds"].mean() + 3 * laptimes["milliseconds"].std()  
outlier_count = (laptimes["milliseconds"] > threshold).sum()
print(f"There are {outlier_count} outliers in this dataset")

There are 681 outliers in this dataset


This dataset includes the time for each lap for each driver for every single race in history. The most meaningful features is time, both in M:S:MS and MS, which seems to be within a reasonable range, with some outliers on the high end. There are 681 outliers in this dataset, which represents 0.12% of the total count. 

Laptime information might be aggregated to compute a more meaningful driver metric (average pace, degradation), this outliers are not going to be of harm for the analysis, and some might go away after the 2014 and forward cutoff is applied. 

### Drivers

In [22]:
drivers = pd.read_csv("data/drivers.csv")
data_profiling("data/drivers.csv")

Features:
 ['driverId', 'driverRef', 'number', 'code', 'forename', 'surname', 'dob', 'nationality', 'url']
Number of rows: 859

Feature: driverId
count    859.000000
mean     430.059371
std      248.213115
min        1.000000
25%      215.500000
50%      430.000000
75%      644.500000
max      860.000000
Name: driverId, dtype: float64

Feature: driverRef
count          859
unique         859
top       hamilton
freq             1
Name: driverRef, dtype: object

Feature: number
count     859
unique     47
top        \N
freq      802
Name: number, dtype: object

Feature: code
count     859
unique     97
top        \N
freq      757
Name: code, dtype: object

Feature: forename
count      859
unique     478
top       John
freq        14
Name: forename, dtype: object

Feature: surname
count        859
unique       800
top       Taylor
freq           5
Name: surname, dtype: object

Feature: dob
count            859
unique           841
top       1918-10-06
freq               2
Name: dob, dtype

This dataset includes driver id, reference name, code, and demographics related to the driver. Datatypes, counts and frequencies are coherent. The number and code columns have missing data in the form of “\N”, these instances will be dealt with after the 2014 cutoff is applied, if necessary. Once the mentioned cut off is applied, the number of drivers will greatly decrease. Out of all the variables, age of the driver might be one of the most meaningful features of the dataset, but apart from that, this dataset’s purpose will be to link the driver to each of their performance once aggregation is performed between the datasets, using the driverid identifier.  

### Races

In [23]:
races = pd.read_csv("data/races.csv")
data_profiling("data/races.csv")

Features:
 ['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time']
Number of rows: 1125

Feature: raceId
count    1125.000000
mean      565.710222
std       328.813817
min         1.000000
25%       282.000000
50%       563.000000
75%       845.000000
max      1144.000000
Name: raceId, dtype: float64

Feature: year
count    1125.000000
mean     1992.703111
std        20.603848
min      1950.000000
25%      1977.000000
50%      1994.000000
75%      2011.000000
max      2024.000000
Name: year, dtype: float64

Feature: round
count    1125.000000
mean        8.579556
std         5.159910
min         1.000000
25%         4.000000
50%         8.000000
75%        13.000000
max        24.000000
Name: round, dtype: float64

Feature: circuitId
count    1125.000000
mean       23.889778
std        19.633527
min         1.000000
25%         9.000000
50%  

This dataset includes raceid, year, round, circuit id, and other race specific columns. Datatypes, counts and frequencies are coherent. The date and time columns for the races have some values in the form “\N”, these instances will be dealt with after the 2014 cutoff is applied, if necessary, since it is likely that date and time was not recorded for these old races. Once the mentioned cut off is applied. Out of all the variables, raceId is a very important one since it will be used to link the races to each driver, year and circuit id will be useful too. The missing values in the FP1/FP2/FP3 times and dates sprint session fields the race “time” column most likely will not be included in the analysis, and their "missigness" does not represent data corruption, which is something good. URL field is a useless column.

### Constructors

In [24]:
constructors = pd.read_csv("data/constructors.csv")
data_profiling("data/constructors.csv")

Features:
 ['constructorId', 'constructorRef', 'name', 'nationality', 'url']
Number of rows: 212

Feature: constructorId
count    212.000000
mean     107.547170
std       61.952685
min        1.000000
25%       54.750000
50%      107.500000
75%      160.250000
max      215.000000
Name: constructorId, dtype: float64

Feature: constructorRef
count         212
unique        212
top       mclaren
freq            1
Name: constructorRef, dtype: object

Feature: name
count         212
unique        212
top       McLaren
freq            1
Name: name, dtype: object

Feature: nationality
count         212
unique         24
top       British
freq           86
Name: nationality, dtype: object

Feature: url
count                                                 212
unique                                                175
top       http://en.wikipedia.org/wiki/Cooper_Car_Company
freq                                                   11
Name: url, dtype: object

Missing values per column:
 constructo

The datatypes in this datasets are coherent, everything seems to be in order. The constructorID column might be the only useful variable in this dataset, this dataset can be used to understand patterns of constructors, and its main purpose will be a connective one. After the 2014 cutoff, many of the constructors will go away 

### Qualifying

In [25]:
quali = pd.read_csv("data/qualifying.csv") 
data_profiling("data/qualifying.csv")

Features:
 ['qualifyId', 'raceId', 'driverId', 'constructorId', 'number', 'position', 'q1', 'q2', 'q3']
Number of rows: 10274

Feature: qualifyId
count    10274.000000
mean      5151.583122
std       2982.227118
min          1.000000
25%       2570.250000
50%       5139.500000
75%       7728.750000
max      10331.000000
Name: qualifyId, dtype: float64

Feature: raceId
count    10274.000000
mean       613.585945
std        426.121490
min          1.000000
25%        140.000000
50%        866.000000
75%        998.000000
max       1133.000000
Name: raceId, dtype: float64

Feature: driverId
count    10274.000000
mean       334.164201
std        387.230090
min          1.000000
25%         16.000000
50%         56.000000
75%        822.000000
max        860.000000
Name: driverId, dtype: float64

Feature: constructorId
count    10274.000000
mean        46.972357
std         72.500393
min          1.000000
25%          4.000000
50%          9.000000
75%         30.000000
max        215.00000

This dataset contains all the link columns for races, constructors and drivers. The values in each column are coherent. The position column is promising since position can be derived into a driver average starting position, which is a determining factor in a race. The missing values in the form “\N” in q2 and q3 are likely from the 2006 and forward since this is the year where this type of qualifying procedure started. The position variable in laptimes and the position here overlap, so that is something that will have to be dealt with.

### Results

In [26]:
results = pd.read_csv("data/results.csv") 
data_profiling("data/results.csv")

Features:
 ['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId']
Number of rows: 26539

Feature: resultId
count    26539.000000
mean     13270.944045
std       7662.590055
min          1.000000
25%       6635.500000
50%      13270.000000
75%      19904.500000
max      26544.000000
Name: resultId, dtype: float64

Feature: raceId
count    26539.000000
mean       546.818644
std        309.943887
min          1.000000
25%        298.000000
50%        527.000000
75%        804.000000
max       1133.000000
Name: raceId, dtype: float64

Feature: driverId
count    26539.000000
mean       274.718791
std        279.565408
min          1.000000
25%         57.000000
50%        170.000000
75%        386.000000
max        860.000000
Name: driverId, dtype: float64

Feature: constructorId
count    26539.000000
mean        49.8330

This dataset contains all the results for every race, it has id columns that will make it possible to link them to each constructor and driver. It includes important variables such as position, points, laps, time, fastest lap, these columns will be of importance for our analysis since they can be derived into more technical characteristics of a driver. Some columns overlap, which will be dealt with too. Missing values in the form of “\N” are present in some of the columns, the qualifying missing values are expected as explained before.

### Pit stops

In [27]:
results = pd.read_csv("data/pit_stops.csv") 
data_profiling("data/pit_stops.csv")

Features:
 ['raceId', 'driverId', 'stop', 'lap', 'time', 'duration', 'milliseconds']
Number of rows: 11030

Feature: raceId
count    11030.000000
mean       976.301995
std         89.381612
min        841.000000
25%        894.000000
50%        967.000000
75%       1056.000000
max       1133.000000
Name: raceId, dtype: float64

Feature: driverId
count    11030.000000
mean       543.368359
std        385.362455
min          1.000000
25%         20.000000
50%        817.000000
75%        832.000000
max        860.000000
Name: driverId, dtype: float64

Feature: stop
count    11030.000000
mean         1.796283
std          1.538376
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         70.000000
Name: stop, dtype: float64

Feature: lap
count    11030.000000
mean        25.327561
std         14.904086
min          1.000000
25%         13.000000
50%         25.000000
75%         36.000000
max         78.000000
Name: lap, dtype: float64

Feature: t

This dataset contains pitstops for every race, it has id columns that will make it possible to link them to each constructor and driver. It contains the stops per race, the lap in which it was done, the duration  and no missing values are included, this dataset has some variables of interest that will be of use for our analysis. After the 2014 cutoff, pit stop strategies became more standardized due to stable regulations, this will make data more symmetrical. Time variables need cleaning.

### Every dataset of interest will now be merged into a single dataset before doing feature engineering. The results dataset will be used as the anchor dataset since it is the one that includes all the identifications.

In [284]:
for file in csv_files:
    file_path = os.path.join(data_folder, file)
    table_name = file.replace('.csv', '')  
    duckdb.register(table_name, duckdb.read_csv(file_path))
    duckdb.sql(f"SELECT * FROM {table_name} LIMIT 5").show()

┌───────────┬─────────────┬──────────────────────┬───┬──────────┬─────────┬───────┬──────────────────────┐
│ circuitId │ circuitRef  │         name         │ … │   lat    │   lng   │  alt  │         url          │
│   int64   │   varchar   │       varchar        │   │  double  │ double  │ int64 │       varchar        │
├───────────┼─────────────┼──────────────────────┼───┼──────────┼─────────┼───────┼──────────────────────┤
│         1 │ albert_park │ Albert Park Grand …  │ … │ -37.8497 │ 144.968 │    10 │ http://en.wikipedi…  │
│         2 │ sepang      │ Sepang Internation…  │ … │  2.76083 │ 101.738 │    18 │ http://en.wikipedi…  │
│         3 │ bahrain     │ Bahrain Internatio…  │ … │  26.0325 │ 50.5106 │     7 │ http://en.wikipedi…  │
│         4 │ catalunya   │ Circuit de Barcelo…  │ … │    41.57 │ 2.26111 │   109 │ http://en.wikipedi…  │
│         5 │ istanbul    │ Istanbul Park        │ … │  40.9517 │  29.405 │   130 │ http://en.wikipedi…  │
├───────────┴─────────────┴──────────

Verifying that the id counts are coherent

In [285]:
duckdb.sql("SELECT COUNT(*) - COUNT(DISTINCT raceId) FROM races;").df()

,(count_star() - count(DISTINCT raceId))
0,0


In [286]:
duckdb.sql("SELECT COUNT(*) - COUNT(DISTINCT driverId) FROM drivers;").df()

,(count_star() - count(DISTINCT driverId))
0,0


In [287]:
duckdb.sql("SELECT COUNT(*) - COUNT(DISTINCT constructorId) FROM constructors;").df()

,(count_star() - count(DISTINCT constructorId))
0,0


Order of aggregation

      → join drivers
      
      → join constructors
      
      → join races
      
          → join circuits

      LATER
      → join pit stops
      
      → join lap times


In [151]:
duckdb.sql("DESCRIBE drivers").df()

,column_name,column_type,null,key,default,extra
0,driverId,BIGINT,YES,NaN,NaN,NaN
1,driverRef,VARCHAR,YES,NaN,NaN,NaN
2,number,VARCHAR,YES,NaN,NaN,NaN
3,code,VARCHAR,YES,NaN,NaN,NaN
4,forename,VARCHAR,YES,NaN,NaN,NaN
5,surname,VARCHAR,YES,NaN,NaN,NaN
6,dob,DATE,YES,NaN,NaN,NaN
7,nationality,VARCHAR,YES,NaN,NaN,NaN
8,url,VARCHAR,YES,NaN,NaN,NaN


In [150]:
duckdb.sql("DESCRIBE results").df()

,column_name,column_type,null,key,default,extra
0,resultId,BIGINT,YES,NaN,NaN,NaN
1,raceId,BIGINT,YES,NaN,NaN,NaN
2,driverId,BIGINT,YES,NaN,NaN,NaN
3,constructorId,BIGINT,YES,NaN,NaN,NaN
4,number,VARCHAR,YES,NaN,NaN,NaN
5,grid,BIGINT,YES,NaN,NaN,NaN
6,position,VARCHAR,YES,NaN,NaN,NaN
7,positionText,VARCHAR,YES,NaN,NaN,NaN
8,positionOrder,BIGINT,YES,NaN,NaN,NaN
9,points,DOUBLE,YES,NaN,NaN,NaN


In [96]:
duckdb.sql("SELECT COUNT(*) FROM results ")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        26539 │
└──────────────┘

In [79]:
#driverId is the link between these two, i will onlly add date of birth to calculate age, and their reference (name)
duckdb.sql( "CREATE TABLE results_drivers AS "
"SELECT r.*, d.driverRef, d.dob "
"FROM results r "
"LEFT JOIN drivers d "
"ON r.driverId = d.driverId;")

In [149]:
duckdb.sql("DESCRIBE results_drivers").df()

,column_name,column_type,null,key,default,extra
0,resultId,BIGINT,YES,NaN,NaN,NaN
1,raceId,BIGINT,YES,NaN,NaN,NaN
2,driverId,BIGINT,YES,NaN,NaN,NaN
3,constructorId,BIGINT,YES,NaN,NaN,NaN
4,number,VARCHAR,YES,NaN,NaN,NaN
5,grid,BIGINT,YES,NaN,NaN,NaN
6,position,VARCHAR,YES,NaN,NaN,NaN
7,positionText,VARCHAR,YES,NaN,NaN,NaN
8,positionOrder,BIGINT,YES,NaN,NaN,NaN
9,points,DOUBLE,YES,NaN,NaN,NaN


In [95]:
duckdb.sql("SELECT COUNT(*) FROM results_drivers;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        26539 │
└──────────────┘

  → join races
  
      → join circuits

  LATER
  → join pit stops
  
  → join lap times

In [288]:
#raceID is the link between races and results_drivers table
#as mentioned before, only year  will be included asa  feature for analysis, circuitId will be included as a link.
duckdb.sql("DESCRIBE races").df()

,column_name,column_type,null,key,default,extra
0,raceId,BIGINT,YES,NaN,NaN,NaN
1,year,BIGINT,YES,NaN,NaN,NaN
2,round,BIGINT,YES,NaN,NaN,NaN
3,circuitId,BIGINT,YES,NaN,NaN,NaN
4,name,VARCHAR,YES,NaN,NaN,NaN
5,time,VARCHAR,YES,NaN,NaN,NaN
6,url,VARCHAR,YES,NaN,NaN,NaN
7,fp1_date,VARCHAR,YES,NaN,NaN,NaN
8,fp1_time,VARCHAR,YES,NaN,NaN,NaN
9,fp2_date,VARCHAR,YES,NaN,NaN,NaN


In [290]:
duckdb.sql(" CREATE TABLE results_drivers_races AS "
"SELECT rd.*, r.year, r.round, r.circuitId "
"FROM races r "
"LEFT JOIN results_drivers rd "
"ON r.raceId = rd.raceId;")

In [147]:
duckdb.sql("DESCRIBE results_drivers_races").df()

,column_name,column_type,null,key,default,extra
0,resultId,BIGINT,YES,NaN,NaN,NaN
1,raceId,BIGINT,YES,NaN,NaN,NaN
2,driverId,BIGINT,YES,NaN,NaN,NaN
3,constructorId,BIGINT,YES,NaN,NaN,NaN
4,number,VARCHAR,YES,NaN,NaN,NaN
5,grid,BIGINT,YES,NaN,NaN,NaN
6,position,VARCHAR,YES,NaN,NaN,NaN
7,positionText,VARCHAR,YES,NaN,NaN,NaN
8,positionOrder,BIGINT,YES,NaN,NaN,NaN
9,points,DOUBLE,YES,NaN,NaN,NaN


In [97]:
duckdb.sql("SELECT COUNT(*) FROM results_drivers_races")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        26550 │
└──────────────┘

In [98]:
duckdb.sql("SELECT raceId, COUNT(*) "
"FROM races "
"GROUP BY raceId "
"HAVING COUNT(*) > 1;")

┌────────┬──────────────┐
│ raceId │ count_star() │
│ int64  │    int64     │
├───────────────────────┤
│        0 rows         │
└───────────────────────┘

In [146]:
duckdb.sql("SELECT r.raceId, r.year, r.round, r.name " 
"FROM races r "
"LEFT JOIN results_drivers rd "
"ON r.raceId = rd.raceId "
"WHERE rd.raceId IS NULL; ").df()

,raceId,year,round,name
0,1135,2024,15,Dutch Grand Prix
1,1136,2024,16,Italian Grand Prix
2,1141,2024,21,São Paulo Grand Prix
3,1143,2024,23,Qatar Grand Prix
4,1144,2024,24,Abu Dhabi Grand Prix
5,1134,2024,14,Belgian Grand Prix
6,1138,2024,18,Singapore Grand Prix
7,1139,2024,19,United States Grand Prix
8,1140,2024,20,Mexico City Grand Prix
9,1137,2024,17,Azerbaijan Grand Prix


In [145]:
duckdb.sql("SELECT COUNT(*) FROM results_drivers_races WHERE year > 2014").df()

,count_star()
0,3999


11 races are lacking a raceId, this is likely because some drivers do not have a recorded result for those races. Reasons behind this could be mismatched data, the race might have not happened by the time the dataset was created since theyre all from 2024, or cancelled races.
Regardless of this, the 11 observation only represent 0.02% of the dataset after the 2014 cutoff, so I will remove them to avoid any issues.

In [102]:
duckdb.sql("DELETE FROM results_drivers_races "
"WHERE driverId IS NULL;")

In [144]:
duckdb.sql("SELECT COUNT(*) FROM results_drivers_races").df()

,count_star()
0,26539


In [143]:
duckdb.sql("SELECT COUNT(*) FROM results_drivers_races WHERE year > 2014").df()

,count_star()
0,3999


In [142]:
duckdb.sql("DESCRIBE circuits").df()

,column_name,column_type,null,key,default,extra
0,circuitId,BIGINT,YES,NaN,NaN,NaN
1,circuitRef,VARCHAR,YES,NaN,NaN,NaN
2,name,VARCHAR,YES,NaN,NaN,NaN
3,location,VARCHAR,YES,NaN,NaN,NaN
4,country,VARCHAR,YES,NaN,NaN,NaN
5,lat,DOUBLE,YES,NaN,NaN,NaN
6,lng,DOUBLE,YES,NaN,NaN,NaN
7,alt,BIGINT,YES,NaN,NaN,NaN
8,url,VARCHAR,YES,NaN,NaN,NaN


circuitId will be the link between circuits and the results_drivers_races table.
Latitude, longitude and altitude will be included due to their physical relevance for racing.  
circuitRef will be included as an identifier, in case it is needed for clarity at any point.

In [141]:
duckdb.sql("CREATE TABLE results_drivers_races_circuits AS "
           "SELECT r.*, c.circuitRef, c.lat, c.lng, c.alt "
           "FROM circuits c "
           "JOIN results_drivers_races r "
           "ON r.circuitId = c.circuitId;"
          )

In [138]:
duckdb.sql("DESCRIBE results_drivers_races_circuits").df()

,column_name,column_type,null,key,default,extra
0,resultId,BIGINT,YES,NaN,NaN,NaN
1,raceId,BIGINT,YES,NaN,NaN,NaN
2,driverId,BIGINT,YES,NaN,NaN,NaN
3,constructorId,BIGINT,YES,NaN,NaN,NaN
4,number,VARCHAR,YES,NaN,NaN,NaN
5,grid,BIGINT,YES,NaN,NaN,NaN
6,position,VARCHAR,YES,NaN,NaN,NaN
7,positionText,VARCHAR,YES,NaN,NaN,NaN
8,positionOrder,BIGINT,YES,NaN,NaN,NaN
9,points,DOUBLE,YES,NaN,NaN,NaN


In [136]:
duckdb.sql("SELECT COUNT(*) FROM results_drivers_races_circuits WHERE year > 2014").df()

,count_star()
0,3999


In [137]:
duckdb.sql("SELECT COUNT(*) "
"FROM results_drivers_races r "
"LEFT JOIN circuits c "
"ON r.circuitId = c.circuitId "
"WHERE c.circuitId IS NULL; "
).df()

,count_star()
0,0


pit stops and lap times will be added later, since they will increase the dataset size by a lot.

In [135]:
duckdb.sql("PRAGMA table_info('results_drivers_races_circuits')").df()

,cid,name,type,notnull,dflt_value,pk
0,0,resultId,BIGINT,False,NaN,False
1,1,raceId,BIGINT,False,NaN,False
2,2,driverId,BIGINT,False,NaN,False
3,3,constructorId,BIGINT,False,NaN,False
4,4,number,VARCHAR,False,NaN,False
5,5,grid,BIGINT,False,NaN,False
6,6,position,VARCHAR,False,NaN,False
7,7,positionText,VARCHAR,False,NaN,False
8,8,positionOrder,BIGINT,False,NaN,False
9,9,points,DOUBLE,False,NaN,False


In [156]:
duckdb.sql("SELECT COUNT(*) AS invalid_milliseconds "
"FROM results_drivers_races_circuits "
"WHERE year >= 2014 "
"AND milliseconds = '\\N'").df()

,invalid_milliseconds
0,2135


In [160]:
duckdb.sql("SELECT COUNT(*) AS invalid_position "
"FROM results_drivers_races_circuits "
"WHERE year >= 2014 "
"AND position = '\\N'").df()

,invalid_position
0,697


In [161]:
duckdb.sql("SELECT COUNT(*) AS invalid_position_and_milliseconds "
"FROM results_drivers_races_circuits "
"WHERE year >= 2014 "
"AND position = '\\N' AND milliseconds = '\\N'" ).df()

,invalid_position
0,695


Many of the drivers do not have a race time, this is most likely due to the fact that many drivers do not finish the race which is a common occurrence in motorsports. When looking deeper in to the data, some of these drivers without a time do have a starting position, which indicates that they indeed had a DNF or a NC. The amount of missing entries where finish position AND milliseconds are null (\N) after the 2014 cutoff is significant (around 17%), these entries can't be removed and must be treated carefully. One way to deal with this might be using a derived feature of finish position instead of time as a feature, and treating these missing values as a DNF of some sorts, that will be assesed later on the feature engineering portion of the project.

position and milliseconds are both in incorrect datatypes, I will transform them into numeric, and treat the \N as NaN values.

In [202]:
duckdb.sql("ALTER TABLE results_drivers_races_circuits "
"ALTER COLUMN position "
"SET DATA TYPE BIGINT "
"USING NULLIF(position, '\\N')::BIGINT;")

In [ ]:
duckdb.sql("PRAGMA table_info('results_drivers_races_circuits')").df().query("name == 'position'")

In [170]:
duckdb.sql("ALTER TABLE results_drivers_races_circuits "
"ALTER COLUMN milliseconds "
"SET DATA TYPE BIGINT "
"USING NULLIF(milliseconds, '\\N')::BIGINT;")

In [171]:
duckdb.sql("PRAGMA table_info('results_drivers_races_circuits')").df().query("name == 'milliseconds'")

,cid,name,type,notnull,dflt_value,pk
12,12,milliseconds,BIGINT,False,NaN,False


In [191]:
duckdb.sql("SELECT COUNT(*) AS zero_count "
"FROM results_drivers_races_circuits "
"WHERE grid = 0;"
)

┌────────────┐
│ zero_count │
│   int64    │
├────────────┤
│       1632 │
└────────────┘

Many of the drivers start from the 0 position, this could indicate that they driver started from the pitlane. 

In [193]:
duckdb.sql("SELECT COUNT(*) AS zero_count "
"FROM results_drivers_races_circuits "
"WHERE laps = 0;"
)

┌────────────┐
│ zero_count │
│   int64    │
├────────────┤
│       2524 │
└────────────┘

In [203]:
duckdb.sql("SELECT COUNT(*) AS zero_count "
"FROM results_drivers_races_circuits "
"WHERE laps = 0 AND position IS NULL;"
)

┌────────────┐
│ zero_count │
│   int64    │
├────────────┤
│       2524 │
└────────────┘

In [196]:
duckdb.sql("SELECT COUNT(*) AS total_rows, "
"COUNT(DISTINCT resultId) AS unique_results "
"FROM results_drivers_races_circuits; ")

┌────────────┬────────────────┐
│ total_rows │ unique_results │
│   int64    │     int64      │
├────────────┼────────────────┤
│      26539 │          26539 │
└────────────┴────────────────┘

No duplicates after merging datasets

In [209]:
duckdb.sql("SELECT COUNT(*) AS zero_lat, "
"FROM results_drivers_races_circuits "
"WHERE lat = 0 ")

┌──────────┐
│ zero_lat │
│  int64   │
├──────────┤
│        0 │
└──────────┘

In [210]:
duckdb.sql("SELECT COUNT(*) AS zero_lng, "
"FROM results_drivers_races_circuits "
"WHERE lng = 0 ")

┌──────────┐
│ zero_lng │
│  int64   │
├──────────┤
│        0 │
└──────────┘

In [211]:
duckdb.sql("SELECT COUNT(*) AS zero_alt, "
"FROM results_drivers_races_circuits "
"WHERE alt = 0 ")

┌──────────┐
│ zero_alt │
│  int64   │
├──────────┤
│      154 │
└──────────┘

0 altitudes are expected since some tracks are at sea level

In [212]:
duckdb.sql("SELECT COUNT(*) AS non_numeric_points "
"FROM results_drivers_races_circuits "
"WHERE TRY_CAST(points AS DOUBLE) IS NULL "
"AND points IS NOT NULL;")

┌────────────────────┐
│ non_numeric_points │
│       int64        │
├────────────────────┤
│                  0 │
└────────────────────┘

All the point entries in the dataset are numeric, this is great since having no missing entries will allow us to do a better analysis of points per season.

Unwanted columns will be dropped on the basis that they are either redundant, or do not bring any value to the study

In [205]:
duckdb.sql("ALTER TABLE results_drivers_races_circuits DROP COLUMN positionText;")
duckdb.sql("ALTER TABLE results_drivers_races_circuits DROP COLUMN number;")
duckdb.sql("ALTER TABLE results_drivers_races_circuits DROP COLUMN time;")
duckdb.sql("ALTER TABLE results_drivers_races_circuits DROP COLUMN fastestLap;")
duckdb.sql("ALTER TABLE results_drivers_races_circuits DROP COLUMN rank;")
duckdb.sql("ALTER TABLE results_drivers_races_circuits DROP COLUMN fastestLapTime;")
duckdb.sql("ALTER TABLE results_drivers_races_circuits DROP COLUMN fastestLapSpeed;")

In [214]:
duckdb.sql("PRAGMA table_info('results_drivers_races_circuits');").df()

,cid,name,type,notnull,dflt_value,pk
0,0,resultId,BIGINT,False,NaN,False
1,1,raceId,BIGINT,False,NaN,False
2,2,driverId,BIGINT,False,NaN,False
3,3,constructorId,BIGINT,False,NaN,False
4,4,grid,BIGINT,False,NaN,False
5,5,position,BIGINT,False,NaN,False
6,6,positionOrder,BIGINT,False,NaN,False
7,7,points,DOUBLE,False,NaN,False
8,8,laps,BIGINT,False,NaN,False
9,9,milliseconds,BIGINT,False,NaN,False


Now that each column is inspected, there are other steps that need to be done to finish with data preparation.

1) The current agreggated table has a row for every driver at every race, for the modeling table, I intend to do driver-season table in which one row represents one driver in one season, which aligns with the goal of predicting the points a driver will earn in a season. Drivers who do a partial season will be included unless there's a more significant reason not to (Such as a driver simply racing for one race due to extrenous situations). Race-normalized features will be preferred to avoid distortion.
2) This will be doable using driverId and the year variable, I intend to initially include total points, DNFs, races entered, average grid position, average finish position, constructor on the season. Altitude and longitude are two very relevant features for race performance, but because these characteristics are circuit specific, there's no way to add it to the driver-season table. Hence, in the feature engineering part of the project, I intend to create a variable that captures this effect and can have explanatory power behind a driver performance, one of the ideas that looks promising is quantify how many high altitude, high latitude races in which a driver participated, ended up in result which falls within their average finish position or better. Average finish rate will be calculated excluding DNFs for this feature only.
3) A DNF flag will be created in order to quantify how many races a driver did not finish, this can help quantify in average how many races a driver does not finish on average. 

In [225]:
duckdb.sql("CREATE TABLE driver_season AS "
"SELECT driverId, year "
"FROM results_drivers_races_circuits "
"WHERE year >= 2014 "
"GROUP BY driverId, year;")

In [227]:
duckdb.sql("PRAGMA table_info('driver_season');").df()

,cid,name,type,notnull,dflt_value,pk
0,0,driverId,BIGINT,False,NaN,False
1,1,year,BIGINT,False,NaN,False


In [235]:
duckdb.sql("SELECT COUNT(*) FROM driver_season;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          244 │
└──────────────┘

In [237]:
duckdb.sql("SELECT COUNT(DISTINCT driverId) AS n_drivers FROM driver_season;")

┌───────────┐
│ n_drivers │
│   int64   │
├───────────┤
│        57 │
└───────────┘

57 unique drivers from the 2014 to 2024 season, 11 seasons in total

In [238]:
duckdb.sql("SELECT COUNT(*) * 1.0 / COUNT(DISTINCT driverId) AS avg_seasons_per_driver FROM driver_season;")

┌────────────────────────┐
│ avg_seasons_per_driver │
│         double         │
├────────────────────────┤
│      4.280701754385965 │
└────────────────────────┘

This number is about right, there are drivers who have raced for multiple years such as Hamilton, Alonso, Ricciardo, and there are a multiple other drivers who raced for only one, two, or three seasons and nothing else.

In [253]:
duckdb.sql("SELECT (*) FROM results_drivers_races_circuits WHERE year > 2014 LIMIT 10;").df()

,resultId,raceId,driverId,constructorId,grid,position,positionOrder,points,laps,milliseconds,statusId,driverRef,dob,year,round,circuitId,circuitRef,lat,lng,alt
0,22538,926,1,131,1,1,1,25.0,58,5514067.0,1,hamilton,1985-01-07,2015,1,1,albert_park,-37.8497,144.968,10
1,22539,926,3,131,2,2,2,18.0,58,5515427.0,1,rosberg,1985-06-27,2015,1,1,albert_park,-37.8497,144.968,10
2,22540,926,20,6,4,3,3,15.0,58,5548590.0,1,vettel,1987-07-03,2015,1,1,albert_park,-37.8497,144.968,10
3,22541,926,13,3,3,4,4,12.0,58,5552263.0,1,massa,1981-04-25,2015,1,1,albert_park,-37.8497,144.968,10
4,22542,926,831,15,10,5,5,10.0,58,5609216.0,1,nasr,1992-08-21,2015,1,1,albert_park,-37.8497,144.968,10
5,22543,926,817,9,6,6,6,8.0,57,NaN,11,ricciardo,1989-07-01,2015,1,1,albert_park,-37.8497,144.968,10
6,22544,926,807,10,13,7,7,6.0,57,NaN,11,hulkenberg,1987-08-19,2015,1,1,albert_park,-37.8497,144.968,10
7,22545,926,828,15,15,8,8,4.0,57,NaN,11,ericsson,1990-09-02,2015,1,1,albert_park,-37.8497,144.968,10
8,22546,926,832,5,7,9,9,2.0,57,NaN,11,sainz,1994-09-01,2015,1,1,albert_park,-37.8497,144.968,10
9,22547,926,815,10,14,10,10,1.0,57,NaN,11,perez,1990-01-26,2015,1,1,albert_park,-37.8497,144.968,10


In [245]:
#Create columns for the modeling table
duckdb.sql("ALTER TABLE driver_season ADD COLUMN total_points DOUBLE;")

duckdb.sql("ALTER TABLE driver_season ADD COLUMN races_entered BIGINT; ")

duckdb.sql("ALTER TABLE driver_season ADD COLUMN dnfs BIGINT; ")

duckdb.sql("ALTER TABLE driver_season ADD COLUMN avg_grid DOUBLE; ")

duckdb.sql("ALTER TABLE driver_season ADD COLUMN avg_finish DOUBLE; ")

duckdb.sql("ALTER TABLE driver_season ADD COLUMN completed_races BIGINT; ")


Populate season-level driver columns

In [271]:
duckdb.sql(
"UPDATE driver_season "
"SET total_points = ( "
"    SELECT SUM(points) "
"    FROM results_drivers_races_circuits r "
"    WHERE r.driverId = driver_season.driverId "
"      AND r.year = driver_season.year "
"      AND points IS NOT NULL "
") "
"WHERE year >= 2014 "
)


In [256]:
duckdb.sql(
    "UPDATE driver_season ds "
    "SET dnfs = agg.dnfs "
    "FROM ( "
    "    SELECT driverId, year, "
    "           SUM(CASE "
    "               WHEN position IS NULL THEN 1 "
    "               ELSE 0 "
    "           END) AS dnfs "
    "    FROM results_drivers_races_circuits "
    "    WHERE year >= 2014 "
    "    GROUP BY driverId, year "
    ") agg "
    "WHERE ds.driverId = agg.driverId "
    "AND ds.year = agg.year "
)

In [257]:
duckdb.sql(
    "UPDATE driver_season ds "
    "SET avg_grid = agg.avg_grid "
    "FROM ( "
    "    SELECT driverId, year, AVG(grid) AS avg_grid "
    "    FROM results_drivers_races_circuits "
    "    WHERE year >= 2014 "
    "    GROUP BY driverId, year "
    ") agg "
    "WHERE ds.driverId = agg.driverId "
    "AND ds.year = agg.year "
)

In [259]:
duckdb.sql(
"UPDATE driver_season ds "
"SET avg_finish = agg.avg_finish_position "
"FROM ( "
"    SELECT driverId, year, "
"           AVG(position) AS avg_finish_position "
"    FROM results_drivers_races_circuits "
"    WHERE position IS NOT NULL "
"    GROUP BY driverId, year "
") agg "
"WHERE ds.driverId = agg.driverId "
"AND ds.year = agg.year "
) 

In [261]:
duckdb.sql(
"UPDATE driver_season ds "
"SET completed_races = agg.completed_races "
"FROM ( "
"    SELECT driverId, year, "
"           SUM(CASE "
"               WHEN milliseconds IS NOT NULL "
"               THEN 1 ELSE 0 "
"           END) AS completed_races "
"    FROM results_drivers_races_circuits "
"    WHERE year >= 2014 "
"    GROUP BY driverId, year "
") agg "
"WHERE ds.driverId = agg.driverId "
"AND ds.year = agg.year "
)

In [263]:
duckdb.sql(
"SELECT driverId, year, COUNT(*) "
"FROM driver_season "
"GROUP BY driverId, year "
"HAVING COUNT(*) > 1 "
)

┌──────────┬───────┬──────────────┐
│ driverId │ year  │ count_star() │
│  int64   │ int64 │    int64     │
├─────────────────────────────────┤
│             0 rows              │
└─────────────────────────────────┘

In [264]:
duckdb.sql(
"SELECT "
"COUNT(*) FILTER (WHERE total_points IS NULL) AS null_points, "
"COUNT(*) FILTER (WHERE races_entered IS NULL) AS null_races, "
"COUNT(*) FILTER (WHERE dnfs IS NULL) AS null_dnfs "
"FROM driver_season "
).df()

,null_points,null_races,null_dnfs
0,0,0,0


In [266]:
duckdb.sql("ALTER TABLE driver_season ADD COLUMN ref VARCHAR; ")

In [268]:
duckdb.sql(
"UPDATE driver_season ds "
"SET ref = agg.driverRef "
"FROM ( "
"    SELECT driverId, year, "
"           MIN(driverRef) AS driverRef "
"    FROM results_drivers_races_circuits "
"    GROUP BY driverId, year "
") agg "
"WHERE ds.driverId = agg.driverId "
"AND ds.year = agg.year "
)

In [273]:
duckdb.sql("ALTER TABLE driver_season ADD COLUMN constructor VARCHAR; ")

In [275]:
duckdb.sql(
"UPDATE driver_season ds "
"SET constructor = agg.constructorId "
"FROM ( "
"    SELECT driverId, year, "
"           FIRST_VALUE(constructorId) OVER (PARTITION BY driverId, year "
"                                            ORDER BY COUNT(*) DESC) AS constructorId "
"    FROM results_drivers_races_circuits "
"    WHERE year >= 2014 "
"    GROUP BY driverId, year, constructorId "
") agg "
"WHERE ds.driverId = agg.driverId "
"AND ds.year = agg.year;  "
)

In [276]:
duckdb.sql("SELECT (*) FROM driver_season;").df()

,driverId,year,total_points,races_entered,dnfs,avg_grid,avg_finish,completed_races,ref,constructor
0,3,2014,317.0,19,2,1.684211,2.529412,16,rosberg,131
1,825,2014,55.0,19,1,8.789474,9.388889,12,kevin_magnussen,1
2,18,2014,126.0,19,1,8.473684,7.500000,13,button,1
3,4,2014,161.0,19,2,6.526316,5.411765,17,alonso,6
4,822,2014,186.0,19,1,6.210526,5.555556,17,bottas,3
...,...,...,...,...,...,...,...,...,...,...
239,154,2020,2.0,15,3,14.133333,14.666667,6,grosjean,210
240,20,2020,33.0,17,2,12.058824,10.400000,9,vettel,6
241,825,2023,3.0,22,4,12.545455,14.888889,7,kevin_magnussen,210
242,857,2023,82.0,22,3,9.636364,9.631579,14,piastri,1


Data preparation is completed to the best of my ability, moving forward I will focus on feature engineering, constructing informative and predictive features from the modeling and source tables based on racing research and domain knowledge.